# LSTM Model for Forex Prediction - Exploratory Analysis
This notebook explores the forex data and builds an initial LSTM model using PyTorch.

Import necessary libraries

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [59]:
import mlflow
import mlflow.pytorch

In [60]:
# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

## 1. Load and Explore Data

Load the data

In [61]:
data_path = "/home/ayaan-ubuntu/DA5402/DA5402-ML-Application/data/prepared/USD_INR_transformed.csv" 
df = pd.read_csv(data_path)

Display basic information

In [ ]:
print(f"Data shape: {df.shape}")
df.head()

### Check for missing values

Check for missing values

In [ ]:
print("Missing values:")
print(df.isnull().sum())

### Exploratory Data Analysis

Statistical summary

In [ ]:
df.describe()

Visualize price and target

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(df['close'], label='Close Price')
plt.plot(df['target'], label='Target (Next Close)', alpha=0.7)
plt.title('Close Price vs Target')
plt.legend()
plt.grid(True)
plt.show()

Visualize technical indicators

In [ ]:
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(df['close'], label='Close')
plt.plot(df['MA5'], label='MA5')
plt.plot(df['MA10'], label='MA10')
plt.plot(df['MA22'], label='MA22')
plt.title('Moving Averages')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(df['close'], label='Close')
plt.plot(df['BB_upper'], label='Upper BB')
plt.plot(df['BB_lower'], label='Lower BB')
plt.title('Bollinger Bands')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(df['RSI'], label='RSI')
plt.axhline(y=70, color='r', linestyle='-', alpha=0.3)
plt.axhline(y=30, color='g', linestyle='-', alpha=0.3)
plt.title('RSI')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Correlation matrix

In [ ]:
# plt.figure(figsize=(12, 10))
# correlation = df.drop(columns=['date']).corr()
# sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5)
# plt.title('Feature Correlation Matrix')
# plt.tight_layout()
# plt.show()

## 2. Prepare Data for LSTM

Define sequence length

In [ ]:
sequence_length = 800

# Extract features and target
features = df.drop(['target', 'date'], axis=1).values
target = df['target'].values

# Normalize features
scaler = StandardScaler()
features

Normalize features

In [ ]:
# Scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Create sequences
X, y = [], []
for i in range(len(features_scaled) - sequence_length):
    X.append(features_scaled[i:i + sequence_length])
    y.append(target[i + sequence_length])

X = np.array(X)
y = np.array(y).reshape(-1, 1)

print(f"X shape: {X.shape}")  # [samples, sequence_length, features]
print(f"y shape: {y.shape}")  # [samples, 1]

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

## 3. Define LSTM Model

In [65]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2, organisation="half"):
        super(LSTMModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.organisation = organisation

        self.lstm_layers = nn.ModuleList()
        current_input_size = input_size
        current_hidden_size = hidden_size

        for i in range(num_layers):
            self.lstm_layers.append(
                nn.LSTM(
                    input_size=current_input_size,
                    hidden_size=current_hidden_size,
                    num_layers=1,
                    batch_first=True,
                    dropout=dropout if i < num_layers-1 else 0
                )
            )

            # Store correct final_hidden_size before changing it
            if i == num_layers - 1:
                final_hidden_size = current_hidden_size

            if organisation == "half":
                current_input_size = current_hidden_size
                current_hidden_size = max(1, current_hidden_size // 2)  # prevent going to 0
            else:
                current_input_size = current_hidden_size

        self.fc = nn.Linear(final_hidden_size, output_size)

    def forward(self, x):
        h = x
        for lstm in self.lstm_layers:
            h0 = torch.zeros(1, h.size(0), lstm.hidden_size).to(x.device)
            c0 = torch.zeros(1, h.size(0), lstm.hidden_size).to(x.device)
            h, _ = lstm(h, (h0, c0))

        out = self.fc(h[:, -1, :])
        return out

## 4. Train Model with MLflow Tracking

Set hyperparameters

In [ ]:
input_size = X_train.shape[2]  # Number of features
hidden_size = 256
num_layers = 2
output_size = 1  # Regression task
dropout = 0.3

batch_size = 32
learning_rate = 0.001
num_epochs = 20

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Start MLflow run

In [ ]:
mlflow.set_experiment("forex_prediction_exploration")

with mlflow.start_run(run_name="LSTM_exploration"):
    # Log model parameters
    mlflow.log_params({
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "dropout": dropout,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "sequence_length": sequence_length
    })
    
    # Lists to store losses for plotting
    train_losses = []
    test_losses = []
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Calculate average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        
        # Evaluate on test set
        model.eval()
        with torch.no_grad():
            test_X, test_y = X_test.to(device), y_test.to(device)
            predictions = model(test_X)
            test_loss = criterion(predictions, test_y).item()
            test_losses.append(test_loss)
            
            # Calculate metrics
            mae = torch.mean(torch.abs(predictions - test_y)).item()
            
            # For directional accuracy
            actual_direction = (test_y[1:] > test_y[:-1]).float()
            pred_direction = (predictions[1:] > predictions[:-1]).float()
            directional_accuracy = torch.mean((actual_direction == pred_direction).float()).item()
        
        # Log metrics to MLflow
        mlflow.log_metrics({
            "train_loss": avg_loss,
            "test_loss": test_loss,
            "mae": mae,
            "directional_accuracy": directional_accuracy
        }, step=epoch)
        
        # Print progress
        if (epoch+1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], "
                f"Train Loss: {avg_loss:.4f}, "
                f"Test Loss: {test_loss:.4f}, "
                f"MAE: {mae:.4f}, "
                f"Dir Acc: {directional_accuracy:.4f}")
    
    # Save model
    mlflow.pytorch.log_model(model, "model")
    
    # Get run ID for reference
    run_id = mlflow.active_run().info.run_id
    print(f"MLflow Run ID: {run_id}")

## 5. Visualize Training Results

Plot training and test loss

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Training and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## 6. Model Evaluation and Predictions

In [ ]:
# Make predictions on test set
model.eval()
with torch.no_grad():
    test_X = X_test.to(device)
    predictions = model(test_X).cpu().numpy()
    actuals = y_test.numpy()

# Plot predictions vs actuals
plt.figure(figsize=(12, 6))
plt.plot(actuals, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Forex Price Prediction - Test Set')
plt.xlabel('Time Steps')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Error analysis
errors = predictions - actuals
plt.figure(figsize=(12, 6))
plt.plot(errors)
plt.title('Prediction Errors')
plt.xlabel('Time Steps')
plt.ylabel('Error')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.title('Error Distribution')
plt.xlabel('Error')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

## 7. Directional Accuracy Analysis

In [ ]:
# Calculate directional movement
actual_direction = np.diff(actuals.flatten()) > 0
pred_direction = np.diff(predictions.flatten()) > 0

# Calculate directional accuracy
directional_accuracy = np.mean(actual_direction == pred_direction)
print(f"Directional Accuracy: {directional_accuracy:.4f}")

# Confusion matrix for direction prediction
confusion = pd.crosstab(
    pd.Series(actual_direction, name='Actual'),
    pd.Series(pred_direction, name='Predicted')
)
print("\nConfusion Matrix (Direction):")
print(confusion)

## 8. MLflow Experiment Tracking

In [ ]:
# Query previous runs
runs = mlflow.search_runs(experiment_names=["forex_prediction_exploration"])
runs[["run_id", "metrics.test_loss", "metrics.directional_accuracy", "params.hidden_size", "params.num_layers"]]

## 9. Hyperparameter Tuning

Data preparation function

In [90]:
def prepare_data(df, sequence_length):
    """
    Load and prepare data for LSTM model
    """
    
    # Extract features and target
    features = df.drop(['target', 'date'], axis=1).values
    target = df['target'].values
    
    # Create sequences
    X, y = [], []
    for i in range(len(features) - sequence_length):
        X.append(features[i:i + sequence_length])
        y.append(target[i + sequence_length])
    
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    
    # Split into training and testing (before scaling!)
    X_train_raw, X_val_raw, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Flatten X for scaling (scale each timestep feature independently)
    num_features = X.shape[2]
    scaler = StandardScaler()
    
    X_train_flat = X_train_raw.reshape(-1, num_features)
    X_val_flat = X_val_raw.reshape(-1, num_features)
    
    X_train_scaled = scaler.fit_transform(X_train_flat).reshape(X_train_raw.shape)
    X_val_scaled = scaler.transform(X_val_flat).reshape(X_val_raw.shape)
    
    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)
    
    return X_train, y_train, X_val, y_val, scaler

Model training function

In [66]:
from itertools import product
from tqdm import tqdm

In [67]:
class CombinedLoss(nn.Module):
    def __init__(self, lambda_direction=0.1):
        super(CombinedLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()  # Predict direction via logits
        self.lambda_direction = lambda_direction  # Weight for directional loss

    def forward(self, predictions, targets):
        # MSE Part
        mse_loss = self.mse(predictions, targets)
        
        # Directional Part
        
        # To compute BCE loss, we need logits (before thresholding):
        pred_diff = predictions[1:] - predictions[:-1]  # raw difference
        true_diff = targets[1:] - targets[:-1]  # raw difference

        # BCE expects labels as 0/1 and inputs as logits
        direction_labels = (true_diff > 0).float()
        direction_logits = pred_diff  # No threshold applied

        directional_loss = self.bce(direction_logits, direction_labels)

        # Total Loss
        total_loss = mse_loss + self.lambda_direction * directional_loss
        return total_loss

In [79]:
def train(X_train, y_train, X_val, y_val, model_params, training_params):
    """
    Train and evaluate LSTM model
    """
    # Unpack parameters
    input_size = X_train.shape[2]  # Number of features
    hidden_size = model_params['hidden_size']
    num_layers = model_params['num_layers']
    output_size = 1  # Regression task
    dropout = model_params['dropout']
    
    batch_size = training_params['batch_size']
    learning_rate = training_params['learning_rate']
    num_epochs = training_params['num_epochs']
    lambda_direction = training_params['lambda_direction']

    # Create DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model, loss function and optimizer
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)
    model.to(device)
    
    criterion = CombinedLoss(lambda_direction=lambda_direction)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    run_name = f"hs{hidden_size}_nl{num_layers}_dr{dropout}_bs{batch_size}_lr{learning_rate}"
    # Start MLflow run
    with mlflow.start_run(run_name=run_name):
        # Log model parameters
        mlflow.log_params({
            "hidden_size": hidden_size,
            "num_layers": num_layers,
            "dropout": dropout,
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "num_epochs": num_epochs,
            "input_size": input_size,
            "sequence_length": X_train.shape[1]
        })
        
        pbar = tqdm(range(num_epochs), desc="Training Progress")
        for epoch in pbar:
            model.train()
            total_loss = 0

            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                # Forward pass
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                # Backward pass and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            # Calculate average loss for the epoch
            avg_loss = total_loss / len(train_loader)
            
            # Evaluate on test set
            model.eval()
            with torch.no_grad():
                val_X, val_y = X_val.to(device), y_val.to(device)
                predictions = model(val_X)
                val_loss = criterion(predictions, val_y).item()
                
                # Calculate metrics
                mae = torch.mean(torch.abs(predictions - val_y)).item()
                
                # For directional accuracy
                actual_direction = (val_y[1:] > val_y[:-1]).float()
                pred_direction = (predictions[1:] > predictions[:-1]).float()
                directional_accuracy = torch.mean((actual_direction == pred_direction).float()).item()
            
            # Log metrics to MLflow
            mlflow.log_metrics({
                "train_loss": avg_loss,
                "val_loss": val_loss,
                "mae": mae,
                "directional_accuracy": directional_accuracy
            }, step=epoch)
            
            # Update tqdm progress bar in-place
            pbar.set_postfix({
                'Train Loss': f"{avg_loss:.4f}",
                'Val Loss': f"{val_loss:.4f}",
                'MAE': f"{mae:.4f}",
                'Dir Acc': f"{directional_accuracy:.4f}"
            })
            
            # Save model
            mlflow.pytorch.log_model(model, "model")
        
        # Get run ID for reference
        run_id = mlflow.active_run().info.run_id
        print(f"MLflow Run ID: {run_id}")
        
        # Return model and metrics for further use
        final_metrics = {
            "val_loss": val_loss,
            "mae": mae,
            "directional_accuracy": directional_accuracy,
            "run_id": run_id
        }
        
    return model, final_metrics

Hyperparameter Tuning

In [69]:
import json

In [82]:
def run_hyperparameter_tuning(df, sequence_length, param_grid):
    """
    Run hyperparameter tuning with MLflow tracking
    """
    
    # Prepare data
    X_train, y_train, X_val, y_val, _ = prepare_data(df, sequence_length)
    
    # Generate all combinations of hyperparameters
    param_combinations = list(product(
        param_grid['hidden_size'],
        param_grid['num_layers'],
        param_grid['dropout'],
        param_grid['batch_size'],
        param_grid['learning_rate'],
        param_grid['num_epochs'],
        param_grid['lambda_direction']
    ))
    
    print(f"Running hyperparameter tuning with {len(param_combinations)} combinations")
    
    # Store results
    results = []
    
    # Run training for each combination
    for hidden_size, num_layers, dropout, batch_size, learning_rate, num_epochs, lambda_direction in param_combinations:
        print(f"Training with: hidden_size={hidden_size}, num_layers={num_layers}, "
              f"dropout={dropout}, batch_size={batch_size}, "
              f"learning_rate={learning_rate}, num_epochs={num_epochs}, lambda={lambda_direction}")
        
        model_params = {
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'dropout': dropout
        }
        
        training_params = {
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'num_epochs': num_epochs,
            'lambda_direction': lambda_direction
        }
        
        # Train and evaluate model
        model, metrics = train(X_train, y_train, X_val, y_val, model_params, training_params)
        
        # Store results
        result = {
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'dropout': dropout,
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'num_epochs': num_epochs,
            'val_loss': metrics['val_loss'],
            'mae': metrics['mae'],
            'directional_accuracy': metrics['directional_accuracy'],
            'run_id': metrics['run_id']
        }
        
        results.append(result)
    
    # Create results dataframe
    results_df = pd.DataFrame(results)
    
    # Find best model based on val loss
    best_idx = results_df['val_loss'].idxmin()
    best_model = results_df.iloc[best_idx]
    
    print(f"Best model found with val_loss: {best_model['val_loss']:.4f}")
    print(f"Best hyperparameters: {best_model.to_dict()}")
    
    # Save results
    os.makedirs('tuning_results', exist_ok=True)
    results_df.to_csv('tuning_results/hyperparameter_results.csv', index=False)
    
    with open('tuning_results/best_params.json', 'w') as f:
        json.dump(best_model.to_dict(), f, indent=4)
    
    return best_model.to_dict()

In [72]:
import warnings
import logging
from tqdm import tqdm

# Suppress warnings from mlflow (and related libraries)
logging.getLogger("mlflow").setLevel(logging.ERROR)

# Alternatively, use warnings module to suppress UserWarnings from mlflow
warnings.filterwarnings("ignore", category=UserWarning, module="mlflow")

In [93]:
mlflow.set_experiment("With half organization - Forex Pred") 

# Define hyperparameter grid
param_grid = {
    'hidden_size': [128, 256, 512],
    'num_layers': [1, 2],
    'dropout': [0.25, 0.5],
    'batch_size': [16, 32, 64],
    'learning_rate': [0.001],
    'num_epochs': [50],
    'lambda_direction': [0.1, 0.2, 0.4]
}

data_path = "/home/ayaan-ubuntu/DA5402/DA5402-ML-Application/data/prepared/USD_INR_transformed.csv" 
df = pd.read_csv(data_path)

sequence_length = 700

# Run hyperparameter tuning
best_params = run_hyperparameter_tuning(df, sequence_length, param_grid)

Running hyperparameter tuning with 108 combinations
Training with: hidden_size=128, num_layers=1, dropout=0.25, batch_size=16, learning_rate=0.001, num_epochs=50, lambda=0.1
Using device: cuda


Training Progress: 100%|██████████| 50/50 [03:51<00:00,  4.62s/it, Train Loss=nan, Val Loss=2221.3213, MAE=47.1301, Dir Acc=0.5625]


MLflow Run ID: 6697f73d460e428389b020fc5e36980f
🏃 View run hs128_nl1_dr0.25_bs16_lr0.001 at: http://localhost:5000/#/experiments/559423165246250756/runs/6697f73d460e428389b020fc5e36980f
🧪 View experiment at: http://localhost:5000/#/experiments/559423165246250756
Training with: hidden_size=128, num_layers=1, dropout=0.25, batch_size=16, learning_rate=0.001, num_epochs=50, lambda=0.2
Using device: cuda


Training Progress: 100%|██████████| 50/50 [03:46<00:00,  4.53s/it, Train Loss=nan, Val Loss=2188.4534, MAE=46.7793, Dir Acc=0.5625]


MLflow Run ID: 8aaa373fc88b4535a2167c982dbbfcc8
🏃 View run hs128_nl1_dr0.25_bs16_lr0.001 at: http://localhost:5000/#/experiments/559423165246250756/runs/8aaa373fc88b4535a2167c982dbbfcc8
🧪 View experiment at: http://localhost:5000/#/experiments/559423165246250756
Training with: hidden_size=128, num_layers=1, dropout=0.25, batch_size=16, learning_rate=0.001, num_epochs=50, lambda=0.4
Using device: cuda


...

Training Progress: 100%|██████████| 50/50 [04:10<00:00,  5.00s/it, Train Loss=nan, Val Loss=3502.3853, MAE=59.1797, Dir Acc=0.5000]


MLflow Run ID: a05c291c24744fa1bd4a070f65d1a323
🏃 View run hs256_nl2_dr0.25_bs32_lr0.001 at: http://localhost:5000/#/experiments/559423165246250756/runs/a05c291c24744fa1bd4a070f65d1a323
🧪 View experiment at: http://localhost:5000/#/experiments/559423165246250756
Training with: hidden_size=256, num_layers=2, dropout=0.25, batch_size=32, learning_rate=0.001, num_epochs=50, lambda=0.4
Using device: cuda


Training Progress:  94%|█████████▍| 47/50 [04:05<00:15,  5.23s/it, Train Loss=nan, Val Loss=3593.0874, MAE=59.9400, Dir Acc=0.4375]


🏃 View run hs256_nl2_dr0.25_bs32_lr0.001 at: http://localhost:5000/#/experiments/559423165246250756/runs/d1759206d14341cf8288678729f46e73
🧪 View experiment at: http://localhost:5000/#/experiments/559423165246250756


KeyboardInterrupt: 

In [ ]:
mlflow.end_run()